In [40]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)

# 1. Load the Datasets
# arabic.csv: The master data
df_arabic = pd.read_csv("source/arabic.csv")

# ai_output.csv: The clean structured data (drug_name, form, dose)
df_ai = pd.read_csv("source/ai.csv")

# old.csv: Loaded just for reference if needed (e.g. checking existing IDs)
df_old = pd.read_csv("source/old.csv")

تنظيف البيانات الأساسية

In [41]:
# Select only the required columns for the Database
required_columns = [
    'id', 'ProductName_EN', 'ProductName_AR', 'new_price', 
    'active_ingredients', 'image_url', 'category', 'company', 
    'form', 'units_per_pack', 'route', 'description'
]

# Create a new dataframe with ONLY these columns
df_arabic = df_arabic[required_columns].copy()

# Data Cleaning

# Clean ProductName_EN (Strip whitespace and convert to string)
df_arabic['ProductName_EN'] = df_arabic['ProductName_EN'].astype(str).str.strip()

# Clean ProductName_AR (Strip whitespace and convert to string)
df_arabic['ProductName_AR'] = df_arabic['ProductName_AR'].astype(str).str.strip()

# Clean new_price (Convert to numeric, coerce errors to NaN, fill with 0)
df_arabic['new_price'] = pd.to_numeric(df_arabic['new_price'], errors='coerce').fillna(0)

# Clean id (Ensure it is an integer)
df_arabic['id'] = pd.to_numeric(df_arabic['id'], errors='coerce').fillna(0).astype(int)

# Clean units_per_pack (Ensure it is an integer, default to 1 if missing)
df_arabic['units_per_pack'] = pd.to_numeric(df_arabic['units_per_pack'], errors='coerce').fillna(1).astype(int)

# Optional: Clean string columns to avoid 'nan' string issues in database
text_columns = ['active_ingredients', 'image_url', 'category', 'company', 'form', 'route', 'description']
for col in text_columns:
    df_arabic[col] = df_arabic[col].astype(str).str.strip().replace(['nan', 'NaN', ''], None)

print(f"Filtered DataFrame shape: {df_arabic.shape}")

Filtered DataFrame shape: (24853, 12)


In [42]:
df_arabic["form"].unique()

array(['tablet', 'syrup', 'spray', 'gel', 'vial', 'capsule', 'piece',
       'cream', 'sachet', 'oral drops', 'suppository', 'tablets',
       'syringe', 'shampoo', 'eye drops', 'effervescent', 'solution',
       'suspension', 'vaginal douche', 'lotion', 'bottle', 'paint',
       'facial wash', 'hair oil', 'pen', 'ampoule', 'eye ointment',
       'ointment', None, 'nasal drops', 'mouth wash', 'serum',
       'ear drops', 'conditioner', 'soap', 'mouth drops', 'powder', 'oil',
       'film', 'Power', 'Cream', 'Gel', 'lozenges', 'Dro', 'patch',
       'tabs.', 'foam', 'Cre', 'caps', 'infusion'], dtype=object)

In [43]:
df_old["form"].unique()

array(['syrup', 'tablet', 'vial', 'sachet', 'cream', 'mouth drops',
       'tablets', 'suppository', 'capsule', 'effervescent', 'suspension',
       'ampoule', 'gel', 'lotion', 'paint', 'facial wash', 'solution',
       'eye drops', 'shampoo', 'hair oil', 'pen', 'eye ointment',
       'syringe', 'ointment', 'oral drops', 'nasal drops', 'bottle',
       'spray', 'mouth wash', 'lozenges', 'piece', 'powder', 'ear drops',
       'vaginal douche', 'oil', 'soap', nan, 'Vial', 'tablet ',
       'capsule ', 'suspension ', 'serum', 'Dro', 'tabs.', 'foam', 'film',
       'conditioner', 'patch', 'caps', 'Power ', 'Cream ', 'Gel ', 'Gel',
       'Cre', 'Cream'], dtype=object)

In [44]:
df_ai["form"].unique()

array(['tablet', 'syrup', 'gel', 'vial', 'capsule', 'Null', 'cream',
       'soap', 'toner', 'mask', 'serum', 'sunscreen', 'sachet', 'spray',
       'belt', 'drops', 'suppository', 'prefilled syringe',
       'effervescent tablet', 'softgel', 'sublingual tablet',
       'conditioner', 'lotion', 'shampoo', 'eye drop', 'suspension',
       'infusion', 'tabs', 'tab', 'eye drops', 'mouth wash', 'supp',
       'caps', 'ear drops', '0', 'cleanser', 'vials', 'oil', 'sachets',
       'nasal spray', 'tablets', 'eff. gr.', 'eff salt',
       'pd for inj./inf./oral', 'susp.', 'pd for inj./inf./inh.',
       'drinkable amp.', 'topical solution', 'capsules', 'powder jar',
       'powder for injection, infusion or inhalation',
       'enteric coated tab', 'sugar coated tab', 'unit dose vial',
       'coated tab', 'caplets', 'solution', 'chewable tablets',
       'chewable tab', 'topical cream', 'extended release tablets',
       'extended release tab', 'oral drops', 'sol', 'patch', 'cap',
       'or

In [45]:
print(df_ai.shape)
print(df_old.shape)

df_ai["original_name"].duplicated().sum()

(26197, 4)
(26201, 11)


np.int64(0)

دمج الداتا فريم

In [46]:
df1_selected = df_ai[['original_name', 'drug_name', 'dose']]
df2_selected = df_old[['tradename', 'form']]

df_merged = pd.merge(
    df1_selected,
    df2_selected,
    left_on='original_name',
    right_on='tradename',
    how='inner'
)

display(df_merged.head(10))
display(df_merged.shape)

,original_name,drug_name,dose,tradename,form
0,1 2 3 (one two three) 20 f.c.tabs.,1 2 3 (one two three),Null,1 2 3 (one two three) 20 f.c.tabs.,tablet
1,1 2 3 (one two three) syrup 120 ml,1 2 3 (one two three),120 ml,1 2 3 (one two three) syrup 120 ml,syrup
2,1 2 3 extra 20 f.c. tab.,1 2 3 extra,Null,1 2 3 extra 20 f.c. tab.,tablet
3,4 wet intimate gel 100 ml,4 wet intimate,100 ml,4 wet intimate gel 100 ml,gel
4,5-fluorouracil-ebewe 250mg/5ml i.v. vial,5-fluorouracil-ebewe,250 mg/ml,5-fluorouracil-ebewe 250mg/5ml i.v. vial,vial
5,5-ht 200 mg 60 caps. (illegal import),5-ht,200 mg,5-ht 200 mg 60 caps. (illegal import),capsule
6,5fu 250mg/5ml vial,5fu,250 mg/ml,5fu 250mg/5ml vial,vial
7,900-900-900ku,900-900-900ku,Null,900-900-900ku,piece
8,Kemstical syrup 120 ml,Kemstical,120 ml,Kemstical syrup 120 ml,syrup
9,Roccatin 20mg,Roccatin,20 mg,Roccatin 20mg,tablet


(26198, 5)

النواقص

In [47]:
# 1. Check for items in df_old MISSING from df_ai
diff_old = pd.merge(
    df_old,
    df_ai[['original_name']],  # We only need the key column from df_ai for checking
    left_on='tradename',
    right_on='original_name',
    how='left',
    indicator=True
)

missing_from_old = diff_old[diff_old['_merge'] == 'left_only'].copy()

# 2. Check for items in df_arabic MISSING from df_ai
diff_arabic = pd.merge(
    df_arabic,
    df_ai[['original_name']],
    left_on='ProductName_EN',
    right_on='original_name',
    how='left',
    indicator=True
)

# Filter rows where the merge indicator is 'left_only'
missing_from_arabic = diff_arabic[diff_arabic['_merge'] == 'left_only'].copy()

# 3. Combine missing items into one DataFrame
names_1 = missing_from_old[['tradename']].rename(columns={'tradename': 'to_process_name'})
names_2 = missing_from_arabic[['ProductName_EN']].rename(columns={'ProductName_EN': 'to_process_name'})

# Concatenate both lists
all_missing = pd.concat([names_1, names_2])

# Drop duplicates (in case the same missing drug is in both df_old and df_arabic)
final_todo_list = all_missing.drop_duplicates().reset_index(drop=True)

print(f"Items missing in df_ai found in df_old: {len(missing_from_old)}")
print(f"Items missing in df_ai found in df_arabic: {len(missing_from_arabic)}")
print(f"Total unique items to process: {len(final_todo_list)}")

print("\n--- Sample of items to process ---")
print(final_todo_list.head())

final_todo_list.to_csv('missing.csv', index=False, encoding='utf-8')

Items missing in df_ai found in df_old: 3
Items missing in df_ai found in df_arabic: 2756
Total unique items to process: 2751

--- Sample of items to process ---
                    to_process_name
0                    lacto g 20 cap
1  optigent 0.3% eye/ear oint. 5 gm
2  optigent 0.3% eye/ear drops 5 ml
3              2hc f.c.t 20 tablets
4       2m whites beegu marin spray


required_columns = [
    'ProductName_EN', 'ProductName_AR', 'price', 
    'active_ingredients#', 'image_url', 'category', 'company', 
    'form', 'units_per_pack', 'route', 'description'
]

التوحيد

In [48]:
# Rename columns in df_old to match df_arabic for easier merging
df_old_renamed = df_old.rename(columns={
    'tradename': 'ProductName_EN',      # Key for matching
    'activeingredient': 'active_ingredients',
    'new_price': 'price_old'            # Rename to avoid conflict, we'll use it to fill gaps later
})

print("✅ df_old columns renamed successfully.")

✅ df_old columns renamed successfully.


In [49]:
# 1. Merge df_arabic with df_old_renamed
# We use 'left' to keep all arabic records.
merged_data = pd.merge(
    df_arabic,
    df_old_renamed,
    on='ProductName_EN',
    how='left', 
    suffixes=('', '_old') # Columns from df_old will have '_old' suffix if name conflicts
)

# 2. Fill missing values in df_arabic columns using data from df_old
# List of columns we want to ensure are complete
cols_to_check = ['active_ingredients', 'company', 'form', 'route']

for col in cols_to_check:
    col_old = f"{col}_old"
    # If the _old column exists, use it to fill NaNs in the main column
    if col_old in merged_data.columns:
        merged_data[col] = merged_data[col].combine_first(merged_data[col_old])

# Special handling for price (fill new_price with price_old if empty)
if 'price_old' in merged_data.columns:
    merged_data['new_price'] = merged_data['new_price'].combine_first(merged_data['price_old'])

print(f"✅ Data merged. Shape: {merged_data.shape}")

✅ Data merged. Shape: (24854, 22)


In [50]:
# Merge with df_ai to get the clean drug_name
final_df = pd.merge(
    merged_data,
    df_ai[['original_name', 'drug_name']], 
    left_on='ProductName_EN',
    right_on='original_name',
    how='left'
)

# Overwrite ProductName_EN with the clean drug_name from df_ai
# If drug_name is NaN (no match in ai), we keep the original ProductName_EN
final_df['ProductName_EN'] = final_df['drug_name'].combine_first(final_df['ProductName_EN'])

print("✅ 'drug_name' from df_ai applied to 'ProductName_EN'.")

✅ 'drug_name' from df_ai applied to 'ProductName_EN'.


In [51]:
# 1. Ensure column is string
final_df['active_ingredients'] = final_df['active_ingredients'].fillna('').astype(str)

# 2. Split by '+' into new columns
ingredients_split = final_df['active_ingredients'].str.split('+', expand=True)

# 3. Rename new columns dynamically (active_ingredients_1, 2, 3...)
ingredients_split.columns = [f'active_ingredients_{i+1}' for i in ingredients_split.columns]

# 4. Join these new columns back to the main dataframe
final_df = pd.concat([final_df, ingredients_split], axis=1)

print(f"✅ Active ingredients split into {len(ingredients_split.columns)} columns.")

✅ Active ingredients split into 27 columns.


In [52]:
# 1. Define static required columns
required_columns = [
    'ProductName_EN', 'ProductName_AR', 'new_price', 
    'image_url', 'category', 'company', 
    'form', 'units_per_pack', 'route', 'description'
]

# 2. Add the dynamic active ingredient columns we just created
# We filter to make sure we only add columns that actually exist
dynamic_ing_cols = [col for col in ingredients_split.columns if col in final_df.columns]
all_target_columns = required_columns + dynamic_ing_cols

# 3. Create the final output DataFrame
df_final_output = final_df[all_target_columns].copy()

# 4. Rename 'new_price' to 'price'
df_final_output = df_final_output.rename(columns={'new_price': 'price'})

# Show results
print("--- Final Output Sample ---")
print(df_final_output.head())

--- Final Output Sample ---
                ProductName_EN               ProductName_AR  price                                    image_url                       category           company    form  units_per_pack        route                                        description active_ingredients_1        active_ingredients_2 active_ingredients_3 active_ingredients_4 active_ingredients_5 active_ingredients_6 active_ingredients_7 active_ingredients_8 active_ingredients_9 active_ingredients_10 active_ingredients_11 active_ingredients_12 active_ingredients_13 active_ingredients_14 active_ingredients_15 active_ingredients_16 active_ingredients_17 active_ingredients_18 active_ingredients_19 active_ingredients_20 active_ingredients_21 active_ingredients_22 active_ingredients_23 active_ingredients_24 active_ingredients_25 active_ingredients_26 active_ingredients_27
0        1 2 3 (one two three)             ون تو ثري 20 قرص   40.0    https://dwaprices.com/upload/55625338.png                    

(24854, 37)
(26197, 4)
